## Постановка задачи
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Разделите выборку на обучающую/проверочную в соотношении 80/20.

Постройте **2 модели** - **kNN по 100** соседей и **множественную логистическую регрессию** - по __наиболее оптимальным наборам параметров (для каждой модели)__, используйте для этого **перекрестную проверку GridSearchCV**.

Проведите предсказание и проверьте качество через каппа-метрику.

Какая модель дала наибольшую точность: логистическая регрессия или kNN ?

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

### Подключение библиотек

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing

### Загрузка данных

In [4]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [5]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)

In [6]:
data = pd.get_dummies(data=data,prefix='Product_Info_2_1')
# df1["Product_Info_2_1"].head()
# df1.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 133 entries, Id to Product_Info_2_1_E
dtypes: float64(18), int64(110), uint8(5)
memory usage: 58.3 MB
None
   Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   2               1              10        0.076923               2   
1   5               1              26        0.076923               2   
2   6               1              26        0.076923               2   
3   7               1              10        0.487179               2   
4   8               1              26        0.230769               2   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0               1               1  0.641791  0.581818  0.148536  ...   
1               3               1  0.059701  0.600000  0.131799  ...   
2               3               1  0.029851  0.745455  0.288703  ...   
3               3               1  0.164179  0.672727  0.205021  ...   
4        

### Общий набор столбцов для расчета

In [7]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [8]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [9]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [10]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [11]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2,
                                         random_state=10)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
33085  1.483398 -1.934570  1.576172  3.925781  0.611816 -0.169434 -1.159180   
14814  0.308838  1.003906  0.441162 -0.213135 -1.634766 -0.169434  0.862305   
1144  -1.218750 -0.220581  1.424805 -1.389648  0.611816 -0.169434 -1.159180   
21288 -0.043610 -0.465576 -0.770020  0.290771  0.611816 -0.169434 -1.159180   
36828  0.003372 -1.690430  0.895020  1.322266  0.611816 -0.169434 -1.159180   

              7         8         9  ...       109      110       111  \
33085  1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   
14814 -1.013672  0.862305  0.100891  ... -0.083679  0.44165 -0.149292   
1144   1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   
21288  1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   
36828  1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   

            112       113      114       115       116       117  Response  
33085  1.

### Логистическая регрессия
Найдем оптимальный набор столбцов и рассчитаем по нему модель логистической регрессии

In [13]:
# Функции взяты из варианта возможного решения

# regression_model
# Создает модель LogisticRegression с заданными параметрами 
#                и обучает ее, если указано при вызове

def regression_model (columns, df, fit=True):
    x = pd.DataFrame(df, columns=columns)
    model = LogisticRegression(max_iter=1000)
    if fit:
        model.fit(x, y = df["Response"])
    return model

# model_score
# подбор гиперпараметров указанной модели по сетке
# Оценка по метрике cohen_kappa_score

def model_score (columns, df_train, model_func):
    x = pd.DataFrame(df_train, columns=columns)
    model = model_func(columns, df_train, False)   # модель должна быть уже обучена
    cv_grid = GridSearchCV(estimator=model, 
                           param_grid={}, 
                           cv=5, 
                           n_jobs=2,
                           scoring=make_scorer(cohen_kappa_score))
    cv_grid.fit(x, df_train["Response"])
    return cv_grid.best_score_

# find_opt_columns
# Поиск оптимального набора параметров модели
# возвращает оптимальный набор колонок и его метрику

def find_opt_columns (data_train, model_func):
    kappa_score_opt = 0
    columns_opt = []
    # для каждой колонки ищем с наилучшей метрикой
    for col in columns_transformed:
        kappa_score = model_score([col], data_train, model_func)
        if kappa_score > kappa_score_opt:
            columns_opt = [col]
            kappa_score_opt = kappa_score
    # для каждой из всех оставшихся проверяем: 
    #     не улучшается ли метрика, если ее добавить в параметры модели
    # возвращаем набор с самой лучшей метрикой и значение его метрики
    for col in columns_transformed:
        if col not in columns_opt:
            columns_opt.append(col)
            kappa_score = model_score(columns_opt, data_train, model_func)
            if kappa_score < kappa_score_opt:
                columns_opt.pop()
            else:
                kappa_score_opt = kappa_score
    return columns_opt, kappa_score_opt

In [14]:
columns_opt_logr, kappa_score_opt = find_opt_columns(data_train,
                                                    regression_model)
model_logr = regression_model(columns_opt_logr, data_train)
print ('model_logr:\n', kappa_score_opt, columns_opt_logr)

model_logr:
 0.3479962343405738 [3, 0, 2, 4, 5, 6, 8, 9, 10, 11, 14, 16, 20, 21, 24, 26, 34, 40, 41, 42, 46, 48, 49, 50, 59, 61, 62, 63, 64, 65, 68, 69, 71, 73, 74, 76, 77, 78, 79, 80, 84, 85, 86, 88, 91, 92, 93, 94, 95, 96, 98, 99, 102, 103, 104, 106, 107, 108, 112, 113, 115, 116, 117]


### kNN
Посчитаем оптимальную модель для kNN

In [15]:
def knn_model (columns, df_train, fit=True):
    y = data_train["Response"]
    x = pd.DataFrame(df_train, columns=columns)
    model = KNeighborsClassifier(n_neighbors=100)
    if fit:
        model.fit(x, y)
    return model

In [16]:
columns_opt_knn, kappa_score_opt = find_opt_columns(data_train,
                                                    knn_model)
model_knn = knn_model(columns_opt_knn, data_train)
print (kappa_score_opt, columns_opt_knn)

0.3657125849625779 [3, 2, 4, 5, 14, 26, 34, 46, 49, 68, 69, 75, 79, 91, 94, 96, 99, 104]


### Предсказание данных и оценка модели

In [17]:
x_test = pd.DataFrame(data_test, columns=columns_opt_logr)
data_test["target_logr"] = model_logr.predict(x_test)
x_test = pd.DataFrame(data_test, columns=columns_opt_knn)
data_test["target_knn"] = model_knn.predict(x_test)

In [18]:
print ("Логистическая регрессия:",
      round(cohen_kappa_score(data_test["target_logr"],
                data_test["Response"], weights="quadratic"), 3))
print ("kNN, 100:",
      round(cohen_kappa_score(data_test["target_knn"],
                data_test["Response"], weights="quadratic"), 3))

Логистическая регрессия: 0.475
kNN, 100: 0.457


### Матрица неточностей

In [19]:
print ("Логистическая регрессия:\n",
      confusion_matrix(data_test["target_logr"], data_test["Response"]))
print ("kNN:\n",
      confusion_matrix(data_test["target_knn"], data_test["Response"]))

Логистическая регрессия:
 [[ 267  155   11   10   67  138   55   32]
 [ 186  264   11    4  136  162   44   14]
 [  10   14   20    4    2    4    0    0]
 [  13   10   24   86    4   54    7    9]
 [ 107  183   11    0  282  139   46   19]
 [ 216  267  117  137  297  945  246  231]
 [ 107   92    3    2   82  210  430   55]
 [ 305  288   12   34  240  627  790 3540]]
kNN:
 [[ 186  115   12   10   44  102   33   16]
 [ 217  307    7    3  108  112   22   14]
 [   0    0    0    0    0    0    0    0]
 [   6    1    5   28    2   21    1    7]
 [  53  132   30    0  285  102    9   11]
 [ 282  302  141  200  332 1135  306  201]
 [ 146  130    6    5  119  278  599  109]
 [ 321  286    8   31  220  529  648 3542]]
